training loss , val loss curve
plotting the distribution
clean data even more
visualize the tree ? 
tree pruning ? 
show results after every technique

In [ ]:
import numpy as np
import pandas as pd 
import tensorflow as tf
from keras.regularizers import l1_l2

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


from keras import models, layers

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import confusion_matrix, accuracy_score, precision_recall_fscore_support, multilabel_confusion_matrix, f1_score
from sklearn.preprocessing import normalize
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras import metrics,optimizers

import matplotlib.pyplot as plt

from keras.backend import clear_session
import re

In [ ]:
X = pd.read_csv('/kaggle/input/movie-plot-title-data/data.csv', index_col = 'Unnamed: 0')['0']
y = pd.read_csv('/kaggle/input/movie-plot-title-data/labels.csv', index_col = 'Unnamed: 0')
y = y.iloc[:,np.where(y.sum(axis = 0) > 800)[0]]

X.drop(np.where(y.sum(axis = 1) == 0)[0], inplace = True)
y.drop(np.where(y.sum(axis = 1) == 0)[0], inplace =True)
genre_list =list(y.columns)


In [ ]:
y.sum(axis = 0)

In [ ]:
# def preprocess_text(sen):
#     # Remove punctuations and numbers
#     sentence = re.sub('[^a-zA-Z.]', ' ', sen)

#     # Single character removal
#     sentence = re.sub(r"\s+[a-zA-Z]\s+", ' ', sentence)

#     # Removing multiple spaces
#     sentence = re.sub(r'\s+', ' ', sentence)

#     return sentence
# X = X.apply(preprocess_text)

'sigmoid' activaation and 'binary_crossentropy' loss

Using count_vec or tfidf with normalization

In [ ]:
def data_generator(data, label, batch_size = 32):
    while True:
        rows = np.random.randint(0, len(data), size = batch_size)
        yield data[rows,:], np.array(label)[rows,:]
def nn_preds_to_classes(preds, threshold = 0.3):
    classes_preds = np.zeros(preds.shape)
    rows, columns = np.where(preds > threshold)
    for i in range(len(rows)):
        classes_preds[rows[i],columns[i]] = 1
    return classes_preds
def eval_nn_model(model, xtest, ytest):
    preds = nn_preds_to_classes(model.predict(xtest))
    
    test_score = f1_score(ytest, preds, average = None)
    return test_score, preds



In [ ]:
max_words = 10000

maxlen = 5000

xtrain, xval, ytrain, yval = train_test_split(X,y, random_state = 0, train_size = 0.8)

tokenizer = Tokenizer(num_words = max_words)
tokenizer.fit_on_texts(xtrain)
xtrain = pad_sequences(tokenizer.texts_to_sequences(xtrain), maxlen = maxlen)
xval = pad_sequences(tokenizer.texts_to_sequences(xval), maxlen = maxlen)



In [ ]:
embeddings = {}
with open('/kaggle/input/glove-global-vectors-for-word-representation/glove.6B.50d.txt') as f : 
    for line in f.readlines():
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings[word] = coefs
f.close()

embeddings_dim = 50
embeddings_matrix = np.zeros((max_words, embeddings_dim))
for word, i in tokenizer.word_index.items():
    if i < max_words:
        embedding_vec = embeddings.get(word)
        if embedding_vec is not None:
            embeddings_matrix[i] = embedding_vec
    

In [ ]:
import keras
lr_schedule = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=1e-1,
    decay_steps=10000,
    decay_rate=0.9)

In [ ]:
tpu = tf.distribute.cluster_resolver.TPUClusterResolver.connect()

# instantiate a distribution strategy
tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)


31 - 33 : 64 lstm, 128 dense, rmsprop ( 37%)
LSTM 64, dropout 0.2, dense 64 1e-3 l2 dropout 0.2 16 epochs : 48%

In [ ]:
with tpu_strategy.scope():    
    model = models.Sequential()
    model.add(layers.Embedding(max_words, embeddings_dim,  input_length = xtrain.shape[1]))
    model.add(layers.LSTM(64))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(64,kernel_regularizer = l1_l2(l1 = 0, l2 = 1e-3)))
    model.add(layers.Dropout(0.2))
    model.add(layers.Dense(len(genre_list), activation = 'sigmoid'))
    model.compile(loss = 'binary_crossentropy',
                  optimizer = optimizers.RMSprop(),
                  metrics = [metrics.AUC()])

    model.summary()

    model.layers[0].set_weights([embeddings_matrix])
    model.layers[0].trainable = False

history = model.fit(xtrain,ytrain,batch_size = 16,
                    epochs = 20,
                    validation_data = (xval,yval)
                  )
val_preds = model.predict(xval)
#     train_preds = model.predict(xtrain)
preds_classes = nn_preds_to_classes(val_preds, threshold = 0.5)
val_score = precision_recall_fscore_support( yval, preds_classes, zero_division = 1)
print(val_score)
print(val_score[0].mean())
print(val_score[1].mean())

print(val_score[2].mean())


In [ ]:
clear_session()

In [ ]:
history_dict = history.history
fig = plt.figure(figsize = (10,10))
plt.plot([i for i in range(20)], history_dict['loss'], label = 'training loss')

plt.plot([i for i in range(20)], history_dict['val_loss'],label = 'validation loss')
plt.legend()

In [ ]:
val_preds = model.predict(xval)
#     train_preds = model.predict(xtrain)
preds_classes = nn_preds_to_classes(val_preds, threshold = 0.5)
val_score = precision_recall_fscore_support( yval, preds_classes, zero_division = 1)
print(val_score)
print(val_score[0].mean())
print(val_score[1].mean())

print(val_score[2].mean())

In [ ]:
fig = plt.figure(figsize = (10,10))
plt.bar(y.columns, val_score[2])